In [1]:
import numpy as np
from scipy import stats
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

import warnings
warnings.filterwarnings('ignore')

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_val_score

In [3]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR

In [4]:
knn = KNeighborsRegressor(n_neighbors=5)
linear = LinearRegression()
sgd = SGDRegressor(max_iter=None, eta0=0.01, penalty='l2', random_state=30)
ridge = Ridge(alpha=1, random_state=30)
lasso = Lasso(alpha=1, random_state=30)
elastic = ElasticNet(alpha=1, l1_ratio=0.5, random_state=30)
dt = DecisionTreeRegressor(max_depth=None, random_state=30)
rf = RandomForestRegressor(n_estimators=100, max_depth=None, random_state=30, n_jobs=-1)
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=30)
svm = SVR(C=1, kernel='rbf', gamma='auto_deprecated')

In [5]:
class change_to_datetime(BaseEstimator, TransformerMixin):
    def __init__(self, column_name=None):
        self.column_name = column_name
        
    def fit(self, df_X, y=None):
        return self
    
    def transform(self, df_X):
        df_X[self.column_name] = pd.to_datetime(df_X[self.column_name])
        return df_X

In [6]:
class divide_datetime(BaseEstimator, TransformerMixin):       
    def fit(self, df_X, y=None):
        return self
    
    def transform(self, df_X):
        df_X["year"] = df_X["datetime"].dt.year
        df_X["month"] = df_X["datetime"].dt.month
        df_X["day"] = df_X["datetime"].dt.day
        df_X["hour"] = df_X["datetime"].dt.hour
        return df_X

In [7]:
class add_dayofweek(BaseEstimator, TransformerMixin):       
    def fit(self, df_X, y=None):
        return self
    
    def transform(self, df_X):
        df_X["dayofweek"] = df_X["datetime"].dt.dayofweek
        return df_X

In [8]:
class one_hot_encoding(BaseEstimator, TransformerMixin):
    def __init__(self, column_name=None, prefix=None):
        self.column_name = column_name
        self.prefix = prefix
        
    def fit(self, df_X, y=None):
        return self
    
    def transform(self, df_X):
        onehotencoding = pd.get_dummies(df_X[self.column_name], prefix = self.prefix)
        return pd.concat([df_X, onehotencoding], axis=1)

In [9]:
class feature_selection(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns
        
    def fit(self, df_X, y=None):
        return self
    
    def transform(self, df_X):
        return df_X[self.columns]

In [10]:
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values):
    # 넘파이로 배열 형태로 바꿔준다.
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
    
    # 예측값과 실제 값에 1을 더하고 로그를 씌워준다.
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)
    
    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 해준다.
    difference = log_predict - log_actual
    # difference = (log_predict - log_actual) ** 2
    difference = np.square(difference)
    
    # 평균을 낸다.
    mean_difference = difference.mean()
    
    # 다시 루트를 씌운다.
    score = np.sqrt(mean_difference)
    
    return score

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

In [11]:
train = pd.read_csv('train.csv')
# train = pd.read_csv("data/train.csv", parse_dates=["datetime"]) 와 같이 불러오면 data type을 변경할 필요 없음
test = pd.read_csv('test.csv')
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [12]:
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [13]:
data = train.drop(['casual', 'registered', 'count'], axis=1)
label = train['count']
data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0


In [14]:
label.head()

0    16
1    40
2    32
3    13
4     1
Name: count, dtype: int64

In [15]:
preparation = make_pipeline(
    change_to_datetime('datetime'),
    divide_datetime(),
    add_dayofweek(),
#     one_hot_encoding('season', 'season'),
    one_hot_encoding('weather', 'weather'),
#     one_hot_encoding('year', 'year'),
    one_hot_encoding('month', 'month'),
    one_hot_encoding('hour', 'hour'),
    one_hot_encoding('dayofweek', 'dayofweek'), 
)

In [16]:
data = preparation.transform(data)
data.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,year,...,hour_21,hour_22,hour_23,dayofweek_0,dayofweek_1,dayofweek_2,dayofweek_3,dayofweek_4,dayofweek_5,dayofweek_6
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,2011,...,0,0,0,0,0,0,0,0,1,0
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,2011,...,0,0,0,0,0,0,0,0,1,0
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,2011,...,0,0,0,0,0,0,0,0,1,0
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,2011,...,0,0,0,0,0,0,0,0,1,0
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,2011,...,0,0,0,0,0,0,0,0,1,0


In [17]:
data.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'year', 'month', 'day', 'hour',
       'dayofweek', 'weather_1', 'weather_2', 'weather_3', 'weather_4',
       'month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
       'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12',
       'hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5', 'hour_6',
       'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11', 'hour_12',
       'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17', 'hour_18',
       'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23', 'dayofweek_0',
       'dayofweek_1', 'dayofweek_2', 'dayofweek_3', 'dayofweek_4',
       'dayofweek_5', 'dayofweek_6'],
      dtype='object')

In [18]:
season = ['season_1', 'season_2', 'season_3', 'season_4']
holiday = ['holiday', 'workingday']
weather = ['weather_1', 'weather_2', 'weather_3', 'weather_4']
month = ['month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6',
         'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12']
hour = ['hour_0', 'hour_1', 'hour_2', 'hour_3', 'hour_4', 'hour_5',
        'hour_6', 'hour_7', 'hour_8', 'hour_9', 'hour_10', 'hour_11',
        'hour_12', 'hour_13', 'hour_14', 'hour_15', 'hour_16', 'hour_17',
        'hour_18', 'hour_19', 'hour_20', 'hour_21', 'hour_22', 'hour_23']
dayofweek = ['dayofweek_0', 'dayofweek_1', 'dayofweek_2', 'dayofweek_3', 'dayofweek_4', 'dayofweek_5', 'dayofweek_6']
num_feature = ['temp', 'humidity', 'windspeed']


feature = holiday + weather + month + hour + dayofweek + num_feature

- season과 month는 같이 사용 X
- temp와 atemp는 같이 사용 X

In [19]:
knn = KNeighborsRegressor(n_neighbors=5)
linear = LinearRegression()
sgd = SGDRegressor(max_iter=None, eta0=0.01, penalty='l2', random_state=30)
ridge = Ridge(alpha=1, random_state=30)
lasso = Lasso(alpha=1, random_state=30)
elastic = ElasticNet(alpha=1, l1_ratio=0.5, random_state=30)
dt = DecisionTreeRegressor(max_depth=None, random_state=30)
rf = RandomForestRegressor(n_estimators=100, max_depth=None, random_state=30, n_jobs=-1)
gb = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=30)
svm = SVR(C=1, kernel='rbf', gamma='auto_deprecated')

In [21]:
models = {
    'knn' : knn,
    'linear' : linear,
    'sgd' : sgd,
    'ridge' : ridge,
    'lasso' : lasso,
    'elastic' : elastic,
    'dt' : dt,
    'rf' : rf,
    'gb' : gb,
    'svm' : svm
}

In [22]:
scores = dict()

In [23]:
for key, model in models.items() : 
    model = make_pipeline(feature_selection(feature), model)
    score = cross_val_score(model, data, label, cv=5, scoring=rmsle_scorer).mean()
    
    scores[key] = score

In [24]:
scores

{'knn': 1.4061397315937176,
 'linear': nan,
 'sgd': nan,
 'ridge': nan,
 'lasso': nan,
 'elastic': nan,
 'dt': 0.7305325712364874,
 'rf': 0.6470521559109927,
 'gb': nan,
 'svm': 1.3897283760418082}

- 스케일러 사용해서 값 비교 해 볼 것